In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

In [ ]:
import serial
import time
import matplotlib.pyplot as plt
import cv2
from pathlib import Path, PurePath
from datetime import date

%matplotlib inline

BAUD_RATE = 9600
NUM_OF_PHOTOS = 47  # One photo for each lamp in the LED strip, grid or
                    # LED ring. If NeoPixel ring is used the maximun
                    # number of Pixels is set to turnonpixel sketch.
BRIGHTNESS_LEVEL = 255
CAM_INDEX = 1  # Often, in a laptop, 0 is the webcam.
TIME_NEXT_PHOTO = 1  # Wait time in seconds to take the next photo.
SS_PATH_SEGMENT = 'SuperScanner'  # SS is the acronym for SuperScanner
SS_PROJECTS_PATH_SEGMENT = 'projects'

# List the serial ports.
serial_ports = !python -m serial.tools.list_ports
for i in range(len(serial_ports)):
    # For each line of the output, remove the whitespace characters at
    # the beginning and the end of the string.
    serial_ports[i] = serial_ports[i].strip()
for item in serial_ports:
    if "ports found" not in item.lower():
        try:
            ser = serial.Serial(item, BAUD_RATE)
            print('<DeviceFound-' + item + '>')
            print('<PC-ArduinoReset>')
            # http://forum.arduino.cc/index.php?topic=38981.0
            # https://stackoverflow.com/questions/21073086/wait-on-arduino-auto-reset-using-pyserial
            ser.dtr = False
            time.sleep(1)
            ser.reset_input_buffer()
            ser.dtr = True
            break
        except serial.SerialException:
            print('<DeviceNotFound>')
cap = cv2.VideoCapture(CAM_INDEX)
try:
    if not cap.isOpened():
        # Try to open the cam again.
        if not cap.open(CAM_INDEX):
            # Force to occurr NameError exception is the cam was not opened.
            print('<CamNotFound-' + str(CAM_INDEX) + '>')
            raise NameError
    else:
        print('<CamFound-' + str(CAM_INDEX) + '>')
    ser
    # Make the project's directory.
    print('<DirectoryMake>')
    home_path = Path.home()
    today_date = str(date.today())
    c = 1
    while True:
        this_project_segment_path = str(c)
        this_project_path = PurePath(home_path,
                                 SS_PATH_SEGMENT,
                                 SS_PROJECTS_PATH_SEGMENT,
                                 today_date,
                                 this_project_segment_path).as_posix()
        try:
            Path(this_project_path).mkdir(parents=True)
            print('<DirectoryCreated>')
            break
        except FileExistsError:
            c = c + 1
    while True:
        try:
            line = ser.readline().strip()
            if b'<ArduinoReady>' in line:
                print(line.decode())
                for i in range(NUM_OF_PHOTOS):
                    outcomingString = str(i + 1) + ',' + str(BRIGHTNESS_LEVEL)
                    ser.write(outcomingString.encode())
                    print('<PC-PixelOn-' + str(i + 1) + '>')
                    pixelOnEvent = '<Arduino-PixelOn-' + str(i + 1) + '>'
                    # https://stackoverflow.com/questions/38645060/what-is-the-equivalent-of-serial-available-in-pyserial
                    while ser.in_waiting == 0:
                        line = ser.readline().strip()
                        if (line == pixelOnEvent.encode()):
                            print(line.decode())
                            break
                    if not cap.isOpened():
                        cap.open(CAM_INDEX)
                    ret, frame = cap.read()
                    img_idx = str(i + 1)
                    for j in range(len(str(NUM_OF_PHOTOS)) - len(str(i))):
                        img_idx = '0' + img_idx
                    filename = 'IMG_' + img_idx +  '.png'
                    file_path = this_project_path + '/' + filename
                    print('<ImageWrite-' + img_idx + '>')
                    cv2.imwrite(file_path, frame)  # Write the photo.
                    # Open the photo below.
                    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                    # Create a figure.
                    plt.figure()
                    try:
                        plt.imshow(img, cmap="Greys_r")
                    except TypeError:
                        print('<PhotoNone-' + str(i) + '>')
                    # Ensure that the photo was taken with light.
                    # time.sleep(TIME_NEXT_PHOTO)
                    # Turn the current Pixel off.
                    outcomingString = str(i + 1) + ',0'
                    ser.write(outcomingString.encode())
                    print('<PC-PixelOff-' + str(i + 1) + '>')
                    pixelOffEvent = '<Arduino-PixelOff-' + str(i + 1) + '>'
                    while ser.in_waiting == 0:
                        line = ser.readline().strip()
                        if (line == pixelOffEvent.encode()):
                            print(line.decode())
                            break
                print(this_project_path)
                raise KeyboardInterrupt
                break
        except KeyboardInterrupt:
            # https://stackoverflow.com/questions/3208566/nested-exceptions
            raise KeyboardInterrupt('1')
        except serial.SerialException:
            cap.release()
            print('<VideoCaptureClose>')
            break
except NameError:
    print('<MicroscopeNotAvailable>')
except KeyboardInterrupt:
    ser.close()
    print('<DeviceClose>')
    cap.release()
    print('<VideoCaptureClose>')